In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
postPorID = {}
with open('subs/userPorPostId.csv') as f:
    for line in f:
        lista = line.split(",")
        postId = lista[2]
        user = lista[1]
        sub = lista[3][:-1]
        postPorID[postId] = (user,sub)

In [3]:
SUB = "the_donald"
FILE = "subs/RC_2017-04."+SUB+".comments.csv"
OUTPUT_FILE = "grafos/"+SUB+".gml"
comments =pd.read_csv(FILE,index_col=0)

In [4]:
print(comments.count())
comments = comments[comments["user"] != "[deleted]"]
comments.set_index('id', inplace=True)

user      200000
id        200000
parent    200000
dtype: int64


In [10]:
postPorID

{'postId': ('user', 'subreddit'),
 '62p82p': ('nobreath', 'cavecomedyradio'),
 '62p82q': ('mplewis1', 'college'),
 '62p82r': ('cooltrain7', 'BlueCorner'),
 '62p82s': ('littlesmol', 'proED'),
 '62p82t': ('RedneckWilly', 'college'),
 '62p82u': ('[deleted]', 'me_irl'),
 '62p82v': ('xKONIGSTIGERx', 'GameSale'),
 '62p82w': ('SativaSammy', 'EDM'),
 '62p82x': ('[deleted]', 'AskFeminists'),
 '62p82y': ('OctoJoey', 'NormalBootsUnited'),
 '62p82z': ('Barefoot_Raphsode', 'BlueCorner'),
 '62p830': ('Luzargx', 'pathofexile'),
 '62p831': ('Exemalti', 'CSgogambling2'),
 '62p832': ('[deleted]', 'nbastreams'),
 '62p833': ('girt-by-sea', 'Neverwinter'),
 '62p834': ('[deleted]', 'RotMG'),
 '62p835': ('pushshift_bot', 'DonaldTrumpWhiteHouse'),
 '62p836': ('droppadabass', 'EDM'),
 '62p837': ('Mobius1337', 'MobiusFF'),
 '62p838': ('hperce', 'toronto'),
 '62p839': ('ranalog', 'analog'),
 '62p83a': ('alle15minuten', 'alle15minuten'),
 '62p83b': ('[deleted]', 'Fireteams'),
 '62p83c': ('Azure_Eagle', 'totalwar'

In [23]:
commentsNulos = "__NOT__"
def getParent(listaComments, commentId):
    tipo, user = commentId.split("_")
    try:
        if tipo == "t1":
            res = listaComments.loc[user]["user"]
        else:
            res = postPorID[user][0]
    except KeyError:
        res = commentsNulos
    return res

In [6]:
userParents = [getParent(x) for x in comments["parent"].tolist()]
comments["padre"] = userParents

In [7]:
comments = comments[comments["padre"] != commentsNulos]
comments = comments[comments["user"] != "[deleted]"]
comments = comments[comments["padre"] != "[deleted]"]

In [8]:
users = comments["user"].unique()
print(len(users),
len(comments))

21418 157012


In [24]:
M=nx.from_pandas_edgelist(comments, "user", 'padre', create_using=nx.MultiDiGraph())
g=nx.DiGraph()
for u,v in M.edges():
    if g.has_edge(u,v):
        g[u][v]['weight'] += 1
    else:
        g.add_edge(u, v, weight=1)
g.remove_edges_from([x for x in g.selfloop_edges()])
g.add_nodes_from(users)
nx.write_gml(g, OUTPUT_FILE)

In [26]:
print(g['rebelde_sin_causa'])

{'wharfthrowaway': {'weight': 5}, 'BerkshireHathaway-': {'weight': 1}, 'want_to_trump': {'weight': 1}, 'TheGettysburgAddress': {'weight': 1}, 'AshleyMagasonDOTcom': {'weight': 3}, 'archangel09': {'weight': 1}, 'I_am_Orlando': {'weight': 9}, 'russian_comrade': {'weight': 1}, 'Plonvick': {'weight': 1}, 'joker68': {'weight': 1}, 'KarmasCominBoys': {'weight': 1}, 'DooDooDoodle': {'weight': 1}, 'number_cruncher_1040': {'weight': 1}, 'fastmandan': {'weight': 1}, 'rolexthewonderdog': {'weight': 3}, 'TrumpIsPatton': {'weight': 2}, '4dNavigator': {'weight': 1}, 'bax101': {'weight': 1}, 'golden430': {'weight': 1}, 'Zadkielred': {'weight': 1}, 'thep1mp': {'weight': 1}, 'Ziggywag': {'weight': 2}, 'politicsmodsrcucks': {'weight': 1}, 'Krakalakalakalak': {'weight': 1}, 'naahbruh': {'weight': 1}, 'messengerofchange': {'weight': 1}, 'NorthJersey908': {'weight': 3}, 'Deplorable_scum': {'weight': 1}, 'BakeRatNoDak': {'weight': 1}, 'h8trdvader': {'weight': 1}, 'isitpedanticenough1': {'weight': 1}, 'lla26

In [33]:
nx.write_gml(g, OUTPUT_FILE)
fd=nx.Graph()
for u,v in g.edges():
    fd.add_edge(u, v)
    if g.has_edge(u,v) and g.has_edge(v,u):
        fd[u][v]['color'] = 'red'
    else:
        fd[u][v]['color'] = 'blue'
nx.write_gml(g, OUTPUT_FILE+".fd.gml")

In [24]:
def crearGrafos(sub):
    FILE = "subs/RC_2017-04."+sub+".comments.csv"
    OUTPUT_FILE = "grafos/"+sub+".gml"
    comments =pd.read_csv(FILE,index_col=0)
    comments = comments[comments["user"] != "[deleted]"]
    comments.set_index('id', inplace=True)
    userParents = [getParent(comments,x) for x in comments["parent"].tolist()]
    comments["padre"] = userParents
    comments = comments[comments["padre"] != commentsNulos]
    comments = comments[comments["user"] != "[deleted]"]
    comments = comments[comments["padre"] != "[deleted]"]
    users = comments["user"].unique()
    M=nx.from_pandas_edgelist(comments, "user", 'padre', create_using=nx.MultiDiGraph())
    g=nx.DiGraph()
    for u,v in M.edges():
        if g.has_edge(u,v):
            g[u][v]['weight'] += 1
        else:
            g.add_edge(u, v, weight=1)
    g.remove_edges_from([x for x in g.selfloop_edges()])
    g.add_nodes_from(users)
    nx.write_gml(g, OUTPUT_FILE)
    fd=nx.Graph()
    for u,v in g.edges():
        fd.add_edge(u, v)
        if g.has_edge(u,v) and g.has_edge(v,u):
            fd[u][v]['color'] = 'red'
        else:
            fd[u][v]['color'] = 'blue'
    nx.write_gml(fd, OUTPUT_FILE+".fd.gml")

In [18]:
listaGrafos = ['android',
'apple',
'argentina',
'askreddit',
'breakingbad',
'conspiracy',
'democrats',
'doctorwho',
'funny',
'gallifrey',
'harrypotter',
'metal',
'nba',
'news',
'nfl',
'politics',
'programming',
'science',
'soccer',
'the_donald',
'worldnews',
'brasil',
'askmen',
'askwomen',
'gameofthrones',
'asoiaf',
'videos',
'pics',
'todayilearned',
'space',
'sports',
'food']

In [25]:
dicGrafos = {}

for gr in listaGrafos:
    crearGrafos(gr)

In [57]:
dicGrafos

{'android': NodeView(('AutoModerator', 'Master_Doe', 'sedp23', 'bballboy699', 'Jobe1105', 'GeneValgene', 'Piece0fCake', 'ahmedsaqlain', 'SmallNuclearRNA', 'hero2007', 'patthagoras', 'xivjae', 'tectoniclakes', 'ruben_fleo', 'RngdZ', 'Bruhmobile', 'sulphide0', 'jonstar7', 'aDisgruntledBadger', 'OurEyesArentReal', '8bitzawad', 'jonnyaas', 'Xpected2', 'bobjimenez_22', 'xMaGnu5MaXx', 'ice-minus', 'brokeninfinity', 'eunderscore', 'OmegaSnowden', 'vishalsharmaofficial', '_dotMonkey', 'Roxasorg13th', 'Uttoktu', 'k4llahz', 'christianabiera', 'Sheraf83', 'mcal24', 'shinoy93', 'Azo3307', 'gcronin', 'Kokialu', 'Fall-Loft', 'asdf987t', 'questiononandroid', 'OG_runandhide05', 'MasterGGM', 'Nyanhunter', 'Emazza', 'extreme__name', 'Rocklee9090', 'Sophia_grtelecs', 'sunnysmation', 'tmayank85', 'harryputtar', 'ronakg', 'ieGod', 'OrganicNebula', 'Alexisandra', 'oneilljstn', '_tracksuitmafia', 'Newdad746', 'toast333', 'fsjja1', 'intelligent_friend', 'jakeeem', 'Jade_Thirlwall', 'Johnytom', 'ejazhus159', '

In [74]:
def commentsSobreUsers(grafo):
    pesos = sum(grafo[u][v]['weight'] for u,v in grafo.edges())
    return pesos/grafo.number_of_nodes()
comSobreUs = [commentsSobreUsers(g1) for g1 in dicGrafos.values()]

print(dicGrafos.keys())
print(comSobreUs)

dict_keys(['android', 'apple', 'argentina', 'askreddit', 'breakingbad', 'conspiracy', 'democrats', 'doctorwho', 'funny', 'gallifrey', 'harrypotter', 'metal', 'nba', 'news', 'nfl', 'politics', 'programming', 'science', 'soccer', 'the_donald', 'worldnews'])
[1.8185847355769231, 1.9450005753077897, 7.137741935483871, 1.81721687505622, 1.3318623124448368, 3.8175828313253013, 1.5039246467817897, 2.254028953837749, 1.3278015703069237, 2.391494002181025, 2.1595517609391677, 1.968566862013852, 4.074588095591509, 1.3649691904525816, 3.0138353902995045, 2.5709962756052143, 1.3545816733067728, 1.1388577827547592, 3.7458651610760545, 6.599669332856696, 1.4321040160398566]


In [76]:
nodos = [g1.number_of_nodes() for g1 in dicGrafos.values()]
clustering = [nx.average_clustering(g1) for g1 in dicGrafos.values()]
densidad = [nx.density(g1) for g1 in dicGrafos.values()]

In [89]:
%matplotlib tk
ax = plt.gca()
ax.plot(nodos, densidad, 'bo')
plt.xlabel('Cantidad de usuarios del subreddit')
plt.ylabel('Densidad del grafo')

Text(0,0.5,'Densidad del grafo')